# Groundwater Level Dossier

In [ ]:
import brodata
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
gld = brodata.gld.GroundwaterLevelDossier.from_bro_id("GLD000000012893")

In [ ]:
gld.to_dict()

In [ ]:
df = gld.observation
df

In [ ]:
f, ax = plt.subplots()
for qualifier in df["qualifier"].unique():
    if pd.isna(qualifier):
        continue
    mask = df["qualifier"] == qualifier
    df.loc[mask, "value"].plot(label=qualifier)
ax.legend();

## Multiple objects

### All measurements of one tube

In [ ]:
df = brodata.gmw.get_tube_observations("GMW000000017757", 1)

In [ ]:
f, ax = plt.subplots()
for qualifier in df["qualifier"].unique():
    if pd.isna(qualifier):
        continue
    mask = df["qualifier"] == qualifier
    df.loc[mask, "value"].plot(label=qualifier)
ax.legend();

### All measurements of one well

In [ ]:
gdf = brodata.gmw.get_observations("GMW000000017757")
gdf = gdf.set_index(["groundwaterMonitoringWell", "tubeNumber"]).sort_index()

In [ ]:
gdf

In [ ]:
f, ax = plt.subplots()
for index in gdf.index.unique():
    observations = [x for x in gdf.loc[[index], "observation"] if not x.empty]
    df = pd.concat(observations).sort_index()
    for qualifier in df["qualifier"].unique():
        if pd.isna(qualifier):
            continue
        mask = df["qualifier"] == qualifier
        label = f"{index[0]}_{index[1]} {qualifier}"
        df.loc[mask, "value"].plot(label=label)
plt.legend();

### All measurements within extent

In [ ]:
extent = [118200, 118400, 439700, 440000]
gdf = brodata.gmw.get_data_in_extent(
    extent=extent, kind="gld", combine=True, as_csv=True
)

In [ ]:
gdf.T

In [ ]:
gdf.plot()

## Observations summary

In [ ]:
brodata.gld.get_observations_summary("GLD000000012893")

## Objects as csv

In [ ]:
df = brodata.gld.get_objects_as_csv("GLD000000012893")
df

In [ ]:
f, ax = plt.subplots()
for qualifier in df["qualifier"].unique():
    if pd.isna(qualifier):
        continue
    mask = df["qualifier"] == qualifier
    df.loc[mask, "value"].plot(label=qualifier)
ax.legend();

## Series as csv

In [ ]:
df = brodata.gld.get_series_as_csv("GLD000000012893")
df